In [159]:
import pandas as pd
import numpy as np

In [160]:
text=pd.read_csv("coupons_ner.csv",header=None)

In [161]:
text

0
0    Save $2.00 ONE Downy Liquid Fabric Conditioner...
1    Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2    Save $2.00 ONE Tide Laundry Detergent (exclude...
3    SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4    $3.00 OFF when you purchase any THREE (3) Pepp...
..                                                 ...
881  SAVE $2.00 when you buy $7 worth of participat...
882  SAVE $5.00 on any Gain scented purchase of $30...
883  SAVE $1.00 when you buy any ONE (1) NEW AXE pr...
884  BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...
885  SAVE $2.00 when you spend $6 on all Suave® pro...

[886 rows x 1 columns]

### Exercise 1 - Returning the FaceValue as Output

In [162]:
import re

In [163]:
text[0][0]

'Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).'

Using Regex to separate the currency/facevalue

Logic - 
Starts with a '$', 1 or more digits, a decimal point(\.), 0 or more digits.

or
Digits followed by cents

or
Single digit after $

or
Decimal after $ followed by 1 or more digits

In [170]:
def find_FV(text):
    if type(text)==str:
        return(re.findall('\$[0-9]+\.[0-9]*|[0-9]+¢|\$[0-9]+|\$\.[0-9]+',text))
    else:
        return([re.findall('\$[0-9]+\.[0-9]*|[0-9]+¢|\$[0-9]+|\$\.[0-9]+',text[0][i]) for i in range(0,len(text))])

In [166]:
FaceValue=find_FV('Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).')
FaceValue

['$2.00']

In [181]:
find_FV("where are my $0.50")

['$0.50']

In [182]:
find_FV("please lend me $.56")

['$.56']

In [176]:
#Extracting the FaceValue from all the entries
FV_all=find_FV(text)

In [178]:
#To check how many entries did not return a FaceValue and their corresponding index
s=0
c=[]
for i in range(0,len(FV_all)):
    if FV_all[i]==[]:
        c.append(i)
        s=s+1

In [179]:
s

273

There are 273 null empty lists in FaceValue.
These are entries which do not contain any FaceValue text

# TASK 2 - Find the product names using NER

# NER

In [183]:
#!pip install spacy

In [184]:
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz


In [187]:
import spacy
import tqdm as tqdm
nlp=spacy.load('en')

In [190]:
d1=nlp("What is his Name Apple")
for t in d1.ents:
    print(t.text, t.start_char, t.end_char, t.label_)

Apple 17 22 ORG


In [191]:
len("Downy Liquid Fabric Conditioner")

31

In [192]:
# Building a corpus for training the model - 
train=[
    ('ONE Downy Liquid Fabric Conditioner'), {'entities':[4,34, 'PRODUCT']},
    ('72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz'), {'entities':[(26,43, 'PRODUCT'),(64,82, 'PRODUCT')]},
    ('Downy Unstopables'), {'entities':[0,16, 'PRODUCT']},
    ('Downy Nature Blends'), {'entities':[0,18, 'PRODUCT']},
    ('Bounce/Downy Sheets'), {'entities':[0,18, 'PRODUCT']},
    ('Bounce/Downy Wrinkle Guard includes '), {'entities':[0,25, 'PRODUCT']},
    ('Included/excludes/whatever Downy Libre Enjuague'), {'entities':[27,46, 'PRODUCT']},
]